<a href="https://colab.research.google.com/github/rahiakela/nlp-research-and-practice/blob/main/text-similarity-works/22_icd_code_descriptions_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ICD Code Descriptions Fine-Tuning

**Reference**:

https://www.pinecone.io/learn/unsupervised-training-sentence-transformers/

##Setup

In [ ]:
%%capture

!pip install datasets
!pip install sentence-transformers

In [ ]:
import datasets

from sentence_transformers import SentenceTransformer, models
from sentence_transformers.datasets import DenoisingAutoEncoderDataset
from sentence_transformers.losses import DenoisingAutoEncoderLoss
from sentence_transformers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

import torch
from torch.utils.data import DataLoader

import re
import pandas as pd
import joblib
import pickle

In [ ]:
%%writefile utils.py
def reverse_code_pattern(p_code):
    orig_code = p_code

    # check for code contains space(" ")
    tmp_code = orig_code.split(" ")
    if len(tmp_code) > 1:
        orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"

    # check for code contains dot(".")
    tmp_code = p_code.split(".")
    if len(tmp_code) > 1:
        orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"

    # check for code contains comma(",")
    tmp_code = p_code.split(",")
    if len(tmp_code) == 2:
        orig_code = f"{tmp_code[0].strip()}.{tmp_code[1].strip()}"
    elif len(tmp_code) == 2:
        orig_code = f"{tmp_code[0].strip()}.{tmp_code[2].strip()}"

    # handle if the first char of code is missing
    alphabats = {"Z": "2", "B": "8", "O": "0", "S": "5", "l": "1", "G": "6", "o": "9", "i": "1"}
    for key, val in alphabats.items():
        # replcae char on 0 index
        if orig_code.find(val) == 0:
            # orig_code = orig_code.replace(val, key)
            orig_code = replacer(orig_code, key, 0)
        # replcae char on 1 index
        if orig_code.find(key) == 1:
            orig_code = replacer(orig_code, val, 1)
            # replcae char on 2 index
            if orig_code.find(key) == 2:
                orig_code = replacer(orig_code, val, 2)
            break

    return orig_code

def replacer(s, newstring, index, nofail=False):
    # raise an error if index is outside of the string
    if not nofail and index not in range(len(s)):
        raise ValueError("index outside given string")

    # if not erroring, but the index is still not in the correct range..
    if index < 0:  # add it to the beginning
        return newstring + s
    if index > len(s):  # add it to the end
        return s + newstring

    # insert the new string between "slices" of the original
    return s[:index] + newstring + s[index + 1:]

Writing utils.py


In [ ]:
from semantic_search import SemanticSearch
import config as cfg

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
!rm -rf data
!unzip data.zip

##ICD Dataset

In [ ]:
icd_description_df = pd.read_csv("data/csv_files/combined_icd_code_and_description_v1.csv")
icd_description_df.head()

,Code,Keyword,SynId,Field_Target
0,001.0,Cholera due to vibrio cholerae,NaN,NaN
1,001.1,Cholera due to vibrio cholerae el tor,NaN,NaN
2,001.9,Cholera unspecified,NaN,NaN
3,002.0,Typhoid fever,NaN,NaN
4,002.1,Paratyphoid fever A,NaN,NaN


In [ ]:
sentences = []
for keyword in icd_description_df["Keyword"]:
  sentences.append(keyword)
sentences[:5]

['Cholera due to vibrio cholerae',
 'Cholera due to vibrio cholerae el tor',
 'Cholera unspecified',
 'Typhoid fever',
 'Paratyphoid fever A']

In [ ]:
# dataset class with noise functionality built-in
train_data = DenoisingAutoEncoderDataset(sentences)

# we use a dataloader as usual
data_loader = DataLoader(train_data, batch_size=4, shuffle=True, drop_last=True)

##Model fine-tuning

In [ ]:
torch.cuda.empty_cache()

###Core BERT Model

In [ ]:
bert_base = models.Transformer("bert-base-uncased")
pooling = models.Pooling(bert_base.get_word_embedding_dimension(), "cls")

model = SentenceTransformer(modules=[bert_base, pooling])

In [ ]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [ ]:
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

In [ ]:
# now ready to begin fine-tuning
model.fit(train_objectives=[(data_loader, loss)],
          epochs=1,
          weight_decay=0,
          scheduler="constantlr",
          optimizer_params={"lr": 3e-5},
          show_progress_bar=True)

In [ ]:
model.save("synodex-icd-bert-base-uncased")

In [ ]:
# save model as pickle file
pickle.dump(model, open("synodex-icd-bert-base-uncased.pickle", 'wb'))

# loaded_model = pickle.load(open("synodex-icd-bert-base-uncased.pickle", "rb"))

In [ ]:
# or create zip file
!zip synodex-icd-bert-base-uncased.zip synodex-icd-bert-base-uncased/*

  adding: synodex-icd-bert-base-uncased/1_Pooling/ (stored 0%)
  adding: synodex-icd-bert-base-uncased/config.json (deflated 48%)
  adding: synodex-icd-bert-base-uncased/config_sentence_transformers.json (deflated 27%)
  adding: synodex-icd-bert-base-uncased/modules.json (deflated 53%)
  adding: synodex-icd-bert-base-uncased/pytorch_model.bin (deflated 8%)
  adding: synodex-icd-bert-base-uncased/README.md (deflated 58%)
  adding: synodex-icd-bert-base-uncased/sentence_bert_config.json (deflated 4%)
  adding: synodex-icd-bert-base-uncased/special_tokens_map.json (deflated 42%)
  adding: synodex-icd-bert-base-uncased/tokenizer_config.json (deflated 43%)
  adding: synodex-icd-bert-base-uncased/tokenizer.json (deflated 71%)
  adding: synodex-icd-bert-base-uncased/vocab.txt (deflated 53%)


###Sentence Transformers model

In [ ]:
# fine-tune on an existing Sentence Transformers model
model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_id)

In [ ]:
loss = DenoisingAutoEncoderLoss(model, tie_encoder_decoder=True)

In [ ]:
# now ready to begin fine-tuning
model.fit(train_objectives=[(data_loader, loss)],
          epochs=1,
          weight_decay=0,
          scheduler="constantlr",
          optimizer_params={"lr": 3e-5},
          show_progress_bar=True)

In [ ]:
model.save("synodex-icd-all-MiniLM-L6-v2")

In [ ]:
# save model as joblib file
joblib.dump(model, "synodex-icd-all-MiniLM-L6-v2.bin")

# loaded_model = joblib.load("synodex-icd-all-MiniLM-L6-v2.bin")

In [ ]:
!zip -r synodex-icd-all-MiniLM-L6-v2.zip synodex-icd-all-MiniLM-L6-v2/*

  adding: synodex-icd-all-MiniLM-L6-v2/1_Pooling/ (stored 0%)
  adding: synodex-icd-all-MiniLM-L6-v2/1_Pooling/config.json (deflated 47%)
  adding: synodex-icd-all-MiniLM-L6-v2/2_Normalize/ (stored 0%)
  adding: synodex-icd-all-MiniLM-L6-v2/config.json (deflated 48%)
  adding: synodex-icd-all-MiniLM-L6-v2/config_sentence_transformers.json (deflated 27%)
  adding: synodex-icd-all-MiniLM-L6-v2/modules.json (deflated 62%)
  adding: synodex-icd-all-MiniLM-L6-v2/pytorch_model.bin (deflated 8%)
  adding: synodex-icd-all-MiniLM-L6-v2/README.md (deflated 53%)
  adding: synodex-icd-all-MiniLM-L6-v2/sentence_bert_config.json (deflated 4%)
  adding: synodex-icd-all-MiniLM-L6-v2/special_tokens_map.json (deflated 42%)
  adding: synodex-icd-all-MiniLM-L6-v2/tokenizer_config.json (deflated 51%)
  adding: synodex-icd-all-MiniLM-L6-v2/tokenizer.json (deflated 71%)
  adding: synodex-icd-all-MiniLM-L6-v2/vocab.txt (deflated 53%)


##Semantic Search

In [ ]:
!mkdir data/models
!cp -r synodex-icd-bert-base-uncased-v1 data/models

In [ ]:
!mkdir -p /home/ocreng/ocrhigh/txt-files
!cp page-7.txt /home/ocreng/ocrhigh/txt-files/

In [ ]:
icd10_code_list = ['Z20.822', 'R05.9', 'E78.3']

In [ ]:
semantic_search = SemanticSearch()
semantic_search.init_corpus_embedding(page_num=7)

In [ ]:
matched_dict = []
for code in icd10_code_list:
  matched_dict.append(semantic_search.get_similarity_score(code))
matched_dict

[{'corpus_id': 11,
  'score': 0.7408628463745117,
  'original_description': 'Contact with and (suspected) exposure to COVID-19',
  'match_description': 'Diagnosis Contact with and suspected exposure COVID19'},
 {'corpus_id': 23,
  'score': 0.636195182800293,
  'original_description': 'Cough, unspecified',
  'match_description': 'Diagnosis Cough'},
 {'corpus_id': 37,
  'score': 0.6819921731948853,
  'original_description': 'Hyperchylomicronemia',
  'match_description': 'Diagnosis Hypertriglyceridemia sporadic'}]

```log
[{'corpus_id': 10,
  'score': 0.681359052658081,
  'original_description': 'Contact with and (suspected) exposure to COVID-19',
  'match_description': 'Diagnosis Contact suspected exposure COVID19 Z20822'},
 {'corpus_id': 21,
  'score': 0.5364741086959839,
  'original_description': 'Cough, unspecified',
  'match_description': 'Diagnosis Cough R059'},
 {'corpus_id': 13,
  'score': 0.4869689643383026,
  'original_description': 'Hyperchylomicronemia',
  'match_description': 'COVID19 Rapid Test Negative Reavionied'}]
```

In [ ]:
print(len(semantic_search.sentence_list))
semantic_search.sentence_list

40


['Patient DOB',
 'Reviewed Tristan Guevara',
 'Order Date',
 'Ordered Muneer behalf',
 'Tristan Guevara',
 'Collected Date Site LABORATORY',
 'Reported Date Requisition',
 'Lab Reviewed COVID19 Rapid Test Accession',
 'Reviewed Tristan Guevara',
 'Notify Tristan Guevara',
 'Diagnosis Contact suspected exposure COVID19 Z20822',
 'Test Name Result Units Normal Range Status',
 'Final',
 'COVID19 Rapid Test Negative Reavionied',
 'Patient DOB',
 'aps Reviewed Bryonna Williams',
 'Order Date Ordered Emily Martinbianco',
 'Collected Date Site LABORATORY',
 'Reported Date Requisition',
 'Lab Reviewed COVID19 Rapid Test Accession',
 'Reviewed Bryonna Williams',
 'Diagnosis Cough R059',
 'Test Name Result Units Normal Range Status',
 'COVID19 Rapid Test Positive Abn Final',
 'Reviewed',
 'Patient DOB',
 'Reviewed Tristan Guevara',
 'Ordered Brenda Bierenga FNPBC',
 'Order Date',
 'behalf Tristan Guevara',
 'Collected Date Site Silver Pine Lab',
 'Reported Date',
 'Lab Reviewed Comprehensive Met